In [9]:
import random
import time
import pandas as pd
import numpy as np
import statsmodels.api as sm


class Rules:
    def __init__(self, bj_payout=1.5, hit_soft17=True, allow_surrender=True, double_after_split=True, resplit_aces=True, hit_split_aces=False, max_hands=4):
        self.bj_payout = bj_payout
        self.hit_soft17 = hit_soft17
        self.allow_surrender = allow_surrender
        self.double_after_split = double_after_split
        self.resplit_aces = resplit_aces
        self.hit_split_aces = hit_split_aces
        self.max_hands = max_hands
        
class Counter:
    def __init__(self):
        values = [2,3,4,5,6,7,8,9,10,11,1]
        self.systems = {
            'hilow': dict(zip(values, [1,1,1,1,1,0,0,0,-1,-1,-1])),
            'tencount': dict(zip(values, [1,1,1,1,1,1,1,1,-2,0,0])),
        }
        self.counts = {system: 0 for system in self.systems}
#         self.observed = dict(zip(values, [0,0,0,0,0,0,0,0,0,0]))
        
    def count(self, card):
#         self.observed[card] += 1
        for system in self.systems:
            self.counts[system] += self.systems[system][card]
    
    def copy(self):
        counter = Counter()
        counter.counts = self.counts.copy()
#         counter.observed = self.observed.copy()
        return counter

class Deck:
    def __init__(self, rules, num_decks, shuffle=True):
        self.rules = rules
        self.num_decks = num_decks
        self.cards = [2,3,4,5,6,7,8,9,10,10,10,10,11] * 4 * self.num_decks
        if shuffle:
            for i in range(5):
                random.shuffle(self.cards)
        self.counter = Counter()
            
    def deal_card(self, count=True):
        card = self.cards.pop()
        if count:
            self.counter.count(card)
        return card
    
    def count(self, card):
        self.counter.count(card)
    
    def cards_remaining(self):
        return len(self.cards)
        
    def decks_remaining(self):
        return self.cards_remaining()/52.
    
    def true_count(self):
        return {k: v/self.decks_remaining() for k, v in self.counter.counts.items()}
    
    def copy(self):
        deck = Deck(self.rules, self.num_decks, shuffle=False)
        deck.cards = self.cards[:]
        deck.counter = self.counter.copy()
        return deck
    
class Hand:
    def __init__(self, deck, cards=[], splitted=False, is_dealer=False, actionable=True, free_hand=False, hand_num=1):
        self.cards = cards[:]
        self.value = 0
        while len(self.cards) < 2:
            if is_dealer and len(self.cards)==0:
                self.cards.append(deck.deal_card(count=False))
            self.cards.append(deck.deal_card())
            
        self.free_hand = free_hand
        self.free_double = False
        self.hand_num = hand_num
        
        self.is_dealer = is_dealer
        self.push = False
        self.splitted = splitted
        self.actionable = actionable
        self.first_action = len(self.cards)==2
        self.pair = self.first_action and (self.cards[0]%10 == self.cards[1]%10) and (hand_num < deck.rules.max_hands)
        assert self.first_action
        
        self.value = sum(self.cards)
        if self.value > 21:
            self.cards[self.cards.index(11)] = 1
            self.value -= 10
#             print(f'{self.cards=}', f'{self.value=}', f'{self.pair=}', f'{self.to_str()=}')
        
        self.soft = 11 in self.cards
        self.busted = self.value > 21
        self.natural = (self.value == 21) and self.first_action and not self.splitted
        self.surrendered = False
        self.doubled = False
    
    def hit(self, deck, double=False, free=False, count=True):
        card = deck.deal_card(count)
        self.cards.append(card)
        self.value += card
        self.first_action = False
        self.pair = False
        self.doubled = double
        self.free_double += double and free
        self.soft = self.soft or card==11
        while self.value > 21 and self.soft:
            self.cards[self.cards.index(11)] = 1
            self.value -= 10
            self.soft = 11 in self.cards
        if self.value > 21:
            self.busted = True
        return card
    
    def split(self, deck):
        assert self.first_action and self.pair and self.hand_num < deck.rules.max_hands
        act = deck.rules.hit_split_aces or self.to_str()!='AA'
        hand1 = Hand(deck, cards=[self.cards[0]], splitted=True, actionable=act, hand_num=self.hand_num + 1)
        hand2 = Hand(deck, cards=[self.cards[1]], splitted=True, actionable=act, hand_num=self.hand_num, free_hand=True)
        return hand1, hand2
    
    def play_hand(self, deck, dealer, action_table, free_table):
        assert not self.is_dealer
        data = {'player': self.to_str(), 'dealer': dealer.to_str(), 'decks_remaining': deck.decks_remaining()}
        data.update({'hilow_act': deck.true_count().copy()['hilow']})
        data.update({'tencount_act': deck.true_count().copy()['tencount']})
        
        if self.natural or dealer.natural or self.busted or self.value==21 or not self.actionable:
            deck.count(dealer.cards[0])
            data['action'] = None
            data['payout'] = resolve_hands([self], dealer, deck.rules)
            return data
        
        action = action_table.loc[self.to_str(), dealer.to_str()]
        player_hands = self.play_player(deck, dealer, action_table, free_table)
        
        if not all(hand.busted for hand in player_hands):
            dealer.play_dealer(deck)
        data['action'] = action
        data['payout'] = resolve_hands(player_hands, dealer, deck.rules)
        data['free_bets'] = count_free_bets(player_hands)
        data['dealer_push'] = dealer.push
        data['dealer_bust'] = dealer.busted and not dealer.push
        return data
    
    def play_player(self, deck, dealer, action_table, free_table, action=None):
        if self.natural or dealer.natural or self.busted or self.value==21 or not (self.actionable or (self.to_str()=='AA' and deck.rules.resplit_aces)):
            return [self]
        if action is None and (self.actionable or (self.to_str()=='AA' and deck.rules.resplit_aces)):
            if self.free_hand:
                action = free_table.loc[self.to_str(), dealer.to_str()]
            else:
                action = action_table.loc[self.to_str(), dealer.to_str()]
        
        if action[0] == 'R':
            if self.first_action and deck.rules.allow_surrender and not self.splitted:
                self.surrendered = True
                return [self]
            else: action = action[1].upper()
        if action[0] == 'P':
            if self.to_str()=='AA' and (self.hand_num >= deck.rules.max_hands or (self.splitted and not deck.rules.resplit_aces)):
                action = 'H'
            elif len(action)==1 or deck.rules.double_after_split:
                assert self.first_action and self.pair and self.hand_num < deck.rules.max_hands
                act = deck.rules.hit_split_aces or self.to_str()!='AA'
                hand1 = Hand(deck, cards=[self.cards[0]], splitted=True, actionable=act, hand_num=self.hand_num + 1)
                h1 = hand1.play_player(deck, dealer, action_table, free_table)
                hand2 = Hand(deck, cards=[self.cards[1]], splitted=True, actionable=act, hand_num=self.hand_num + len(h1), free_hand=True)
                h2 = hand2.play_player(deck, dealer, action_table, free_table)
                return h1 + h2
            else: action = action[1].upper()
        if action[0] == 'D':
            if self.first_action and (deck.rules.double_after_split or not self.splitted):
                card = self.hit(deck, double=True)
                return [self]
            elif len(action)==1:
                action = 'H'
            else: action = action[1].upper()
        if action == 'FD':
            if self.first_action:
                card = self.hit(deck, double=True, free=True)
                return [self]
            else:
                action = 'H'
        if action == 'H':
            card = self.hit(deck)
            return self.play_player(deck, dealer, action_table, free_table)
        elif action == 'S':
            return [self]
        
        print(action)
        raise Exception
    
    def play_dealer(self, deck):
        assert self.is_dealer
        deck.count(self.cards[0])
        while self.value < 17 or (deck.rules.hit_soft17 and self.value==17 and self.soft):
            card = self.hit(deck)
        self.push = self.value==22
        return self
    
    def copy(self):
        assert len(self.cards)>=2
        return Hand(None, self.cards, splitted=self.splitted, is_dealer=self.is_dealer, actionable=self.actionable)
        
    def card2str(self, card):
        return {11:'A', 10:'T', 1:'A'}.get(card, str(card))
        
    def to_str(self):
        if self.is_dealer:
            return self.card2str(self.cards[1])
        if self.value == 21:
            return '21'
        if self.pair and self.first_action:
            return self.card2str(self.cards[0])*2
        if self.soft:
            return 'A' + self.card2str(self.value-11)
        return str(self.value)
        
def resolve_hands(player_hands, dealer, rules):
    total = 0
    for hand in player_hands:
        payout = 0
        if hand.natural:
            payout = rules.bj_payout if not dealer.natural else 0
        elif dealer.natural:
            payout = 0 if hand.free_hand else -1
        elif hand.surrendered:
            payout = -.5
        elif hand.busted:
            payout = (0 if hand.free_hand else -1) - (1 if hand.doubled and not hand.free_double else 0)
        elif dealer.push:
            payout = 0
        elif dealer.busted:
            payout = 2 if hand.doubled else 1
        elif hand.value > dealer.value:
            payout = 2 if hand.doubled else 1
        elif hand.value < dealer.value:
            payout = (0 if hand.free_hand else -1) - (1 if hand.doubled and not hand.free_double else 0)
        total += payout
    return total

def count_free_bets(player_hands):
    return sum([hand.free_hand*1 + hand.free_double*1 for hand in player_hands])

class FreeBetBlackjack:
    def __init__(self, bj_payout=1.5, hit_soft17=True, allow_surrender=False, double_after_split=True, resplit_aces=True, hit_split_aces=False):
        self.rules = Rules(bj_payout, hit_soft17, allow_surrender, double_after_split, resplit_aces, hit_split_aces)
        self.action_table = pd.read_csv('/Users/alex/Documents/Gambling/Blackjack/decision_table - FB - real money.csv', index_col=0).fillna('')
        self.free_table = pd.read_csv('/Users/alex/Documents/Gambling/Blackjack/decision_table - FB - free hand.csv', index_col=0).fillna('')
        
    # play a single deck of cards and record data on each hand
    def play_single_deck(self, num_hands=1):
        num_decks = 1
        self.deck = Deck(self.rules, num_decks)
        self.data = []
        for i in range(6-num_hands):
            payout = self.play_hand(self.deck)
            self.data.append(payout)
        return self.data
            
    def play_multi_deck(self, decks=2, penetration=1.25):
        num_decks = decks
        self.deck = Deck(self.rules, num_decks)
        self.data = []
        while self.deck.decks_remaining() > self.deck.num_decks - penetration:
            payout = self.play_hand(self.deck)
            if type(payout) is list:
                print(payout)
            self.data.append(payout)
        return self.data
        
    # play a hand out of the deck, modify data with results, return deck with cards removed
    def play_hand(self, deck, split_hands=1, player=None, dealer=None):
        count = deck.true_count().copy()
        player = Hand(deck)
        dealer = Hand(deck, is_dealer=True)
        
#         player_hands = player.play_player(deck, dealer, self.action_table)
#         dealer = dealer.play_dealer(deck)
        
        data = player.play_hand(deck, dealer, self.action_table, self.free_table)
        data.update({'hilow_bet': count['hilow'], 'tencount_bet': count['tencount']})
        return data


In [2]:
game = FreeBetBlackjack(bj_payout=1.5, hit_soft17=True, allow_surrender=False)
decks = 6
penetration = 4.75


i = 1
max_i = 1
df = []
t = time.time()
while i <= max_i:
    data = game.play_multi_deck(decks, penetration)
    df += data
    if len(df) > 1e7:
        print(i, time.time() - t)
#         print(payouts)
#         print(pd.DataFrame(payouts))
        df = pd.DataFrame(df)
        df.to_csv(f'/Users/alex/Documents/Gambling/Blackjack/6deck_freebet_3to2_H17_{i}.csv')
        i += 1
        t = time.time()
        if i <= max_i:
            df = []
# sum(payouts), len(payouts), sum(payouts)/len(payouts), (sum([p**2 for p in payouts])**.5/len(payouts))

1 505.4875671863556


## Free Bet Analysis

In [186]:
if len(df)==0:
    df = pd.read_csv(f'/Users/alex/Documents/Gambling/Blackjack/6deck_freebet_3to2_H17_1.csv', index_col=0)
df

,player,dealer,decks_remaining,hilow,action,payout,free_bets,dealer_push,dealer_bust
0,TT,T,5.923077,-0.506494,None,-1.0,NaN,NaN,NaN
1,21,4,5.846154,-0.855263,None,1.5,NaN,NaN,NaN
2,19,5,5.769231,-0.866667,S,-1.0,0.0,False,False
3,12,7,5.673077,-0.705085,H,-1.0,0.0,False,False
4,99,5,5.538462,-0.180556,P,2.0,1.0,False,True
...,...,...,...,...,...,...,...,...,...
10000014,11,T,1.653846,-6.651163,FD,-1.0,1.0,False,False
10000015,14,T,1.557692,-7.061728,H,1.0,0.0,False,False
10000016,14,2,1.423077,-5.621622,S,-1.0,0.0,False,False
10000017,19,4,1.288462,-3.104478,S,0.0,0.0,False,False


In [26]:
df.loc[df['action'].isin(['P'])]

,Unnamed: 0,player,dealer,decks_remaining,hilow,action,payout,free_bets,dealer_push,dealer_bust,pot_of_gold,hilow_int
54,54,77,9,4.923077,-1.218750,P,-1.0,2,False,False,12,-1
72,72,77,8,2.942308,0.000000,P,2.0,2,False,False,12,0
84,84,88,T,1.615385,-0.619048,P,-3.0,1,False,False,3,-1
88,88,22,7,5.826923,0.171617,P,0.0,2,True,False,12,0
113,113,88,T,3.211538,-1.868263,P,-1.0,1,False,False,3,-2
...,...,...,...,...,...,...,...,...,...,...,...,...
9999874,9999874,66,9,3.980769,-3.014493,P,2.0,2,False,True,12,-3
9999896,9999896,88,8,1.346154,-1.485714,P,2.0,1,False,True,3,-1
9999916,9999916,88,T,4.038462,-0.990476,P,2.0,1,False,True,3,-1
9999922,9999922,88,2,3.384615,-1.477273,P,-1.0,1,False,False,3,-1


In [187]:
df['payout'].mean()

-0.016139419335103263

In [3]:
pot_of_gold = pd.Series({
    0: -1,
    1: 3,
    2: 12,
    3: 30,
    4: 50,
    5: 100,
    6: 100,
    7: 100,
})
df['free_bets'] = df['free_bets'].fillna(0).astype(int)
df['pot_of_gold'] = pot_of_gold[df['free_bets']].values
df['pot_of_gold'].mean()

-0.12135831796252305

In [5]:
df['hilow_int'] = df['hilow_bet'].round(0).astype(int)
df.groupby('hilow_int')['payout'].mean()

hilow_int
-25   -1.000000
-22    0.200000
-21   -0.333333
-20    0.136364
-19   -0.555556
-18    0.039474
-17   -0.067935
-16   -0.141927
-15   -0.031085
-14   -0.024194
-13   -0.017709
-12   -0.038004
-11   -0.052835
-10   -0.035635
-9    -0.042257
-8    -0.046418
-7    -0.035553
-6    -0.036277
-5    -0.026455
-4    -0.027818
-3    -0.022832
-2    -0.020849
-1    -0.017159
 0    -0.014951
 1    -0.011599
 2    -0.008237
 3    -0.005758
 4    -0.003419
 5     0.001500
 6     0.002187
 7     0.011911
 8     0.008713
 9     0.009832
 10    0.023152
 11    0.030321
 12   -0.022068
 13   -0.019361
 14    0.074096
 15    0.041966
 16    0.031088
 17    0.163265
 18   -0.119048
 19   -0.250000
 20   -0.428571
 21    0.200000
 22   -1.000000
 23   -1.000000
Name: payout, dtype: float64

In [7]:
X = sm.add_constant(df['hilow_bet'])
Y = df['payout']
r = sm.OLS(Y, X).fit()
r.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 payout   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     548.1
Date:                Sat, 08 Oct 2022   Prob (F-statistic):          3.36e-121
Time:                        01:27:27   Log-Likelihood:            -1.4859e+07
No. Observations:            10000015   AIC:                         2.972e+07
Df Residuals:                10000013   BIC:                         2.972e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0146      0.000    -43.078      0.000      -0.015      -0.014
hilow_bet      0.0031      0.000     23.411      0.000       0.003       0.003
==============================================================================
Omnibus:                  2128426.378   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           810509.470
Skew:                           0.512   Prob(JB):                         0.00
Kurtosis:                       2.053   Cond. No.                         2.60
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [47]:
(df.groupby('free_bets')['pot_of_gold'].count() / len(df)).apply(lambda x: '%.8f' % x)

free_bets
0    0.83814782
1    0.14539589
2    0.01296180
3    0.00297960
4    0.00043200
5    0.00007510
6    0.00000750
7    0.00000030
Name: pot_of_gold, dtype: object

In [190]:
(df.groupby('free_bets')['pot_of_gold'].count() / len(df)).apply(lambda x: '%.8f' % x)

free_bets
0    0.83793681
1    0.14517302
2    0.01338047
3    0.00299249
4    0.00043850
5    0.00007190
6    0.00000640
7    0.00000040
Name: pot_of_gold, dtype: object

In [43]:
df.groupby('hilow_int')['pot_of_gold'].mean()

hilow_int
-23   -1.000000
-22   -1.000000
-21    0.133333
-20   -0.411765
-19   -0.349206
-18   -0.260000
-17   -0.178683
-16   -0.188172
-15   -0.196220
-14   -0.031770
-13   -0.171268
-12   -0.161111
-11   -0.186183
-10   -0.147059
-9    -0.145170
-8    -0.191600
-7    -0.167484
-6    -0.188142
-5    -0.195299
-4    -0.198925
-3    -0.209539
-2    -0.220490
-1    -0.235123
 0    -0.132028
 1     0.006646
 2    -0.027699
 3    -0.033006
 4    -0.048115
 5    -0.059378
 6    -0.047054
 7    -0.066123
 8    -0.095368
 9    -0.089769
 10   -0.091054
 11   -0.061136
 12   -0.014777
 13   -0.058709
 14   -0.023070
 15   -0.244643
 16    0.082707
 17    0.151724
 18    0.216216
 19   -0.652174
 20   -0.200000
 21   -1.000000
 22   -1.000000
Name: pot_of_gold, dtype: float64

In [44]:
X = sm.add_constant(df['hilow_int'])
Y = df['pot_of_gold']
r = sm.OLS(Y, X).fit()
r.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            pot_of_gold   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     3824.
Date:                Fri, 07 Oct 2022   Prob (F-statistic):               0.00
Time:                        20:23:38   Log-Likelihood:            -2.4939e+07
No. Observations:            10000001   AIC:                         4.988e+07
Df Residuals:                 9999999   BIC:                         4.988e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1235      0.001   -133.081      0.000      -0.125      -0.122
hilow_int      0.0213      0.000     61.841      0.000       0.021       0.022
==============================================================================
Omnibus:                 15567607.822   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      14331760143.052
Skew:                           9.784   Prob(JB):                         0.00
Kurtosis:                     187.427   Cond. No.                         2.70
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""